In [1]:
### Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import time
import os
import sys
import sklearn
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer as CountVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.python.client import device_lib

#import nltk

In [2]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils

In [3]:
corpus = utils.load_corpus()

In [4]:
'''songs = utils.split_text(corpus)
all_songs = []
for song in songs:
    all_songs.append(utils.split_song(song))'''

'songs = utils.split_text(corpus)\nall_songs = []\nfor song in songs:\n    all_songs.append(utils.split_song(song))'

In [5]:
words = utils.preprocess_text(corpus, fun_list = [utils.to_lower, utils.decontraction, utils.remove_punct], keep = '\<|\>')
words = re.sub('\n',' \n ', words)
words = re.split(' +', words)

In [6]:
word_count = Counter(words)
index_to_vocab = {i: k for i, k in enumerate(word_count.keys())}
vocab_to_index = {k: i for i, k in enumerate(word_count.keys())}

In [7]:
songs = ' '.join(words)
songs = songs.split(' \n \n <eos> \n \n ')
songs = [song.split(' ') for song in songs]

In [12]:
window_len = 15
vocab_size = len(word_count)
embedding_dim = 128
rnn_dim = 256

NotImplementedError: Cannot convert a symbolic Tensor (encoder/strided_slice_2:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.15.0
    Uninstalling absl-py-0.15.0:
      Successfully uninstalled absl-py-0.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstal

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\quekh\\AppData\\Local\\Temp\\pip-uninstall-_z0vwh3a\\python\\_pywrap_mlir.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
# Encoder
encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(
    encoder_input
)

# Return state in addition to output
rnn = layers.SimpleRNN(embedding_dim, return_state=True, name="encoder")(
    encoder_embedded
)

# Decoder
decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(
    decoder_input
)

# Pass the 2 states to a new RNN, as initial state
decoder_output = layers.SimpleRNN(embedding_dim, name="decoder")(
    decoder_embedded, initial_state=[encoder_state]
)
output = layers.Dense(vocab_size)(decoder_output)

model = keras.Model([encoder_input, decoder_input], output)
model.summary()